In [1]:
## Libraries

# base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random


# preprocessing
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

# data
from sklearn.datasets import load_digits

# clustering
from sklearn.cluster import DBSCAN, KMeans

#external indices
from sklearn.metrics.cluster import adjusted_rand_score, contingency_matrix

# runtime and run tracking
import time
from datetime import datetime

# pathing
from pathlib import Path
import os
import sys



___

### Start Timer

In [2]:
# track runtime
start = time.time()

___

### Define File Paths

In [3]:
# define path
nb_path = Path(os.getcwd())
print(nb_path)
path = str(nb_path.parent)
print(path)

# path to figs folder
figs_path = path + '/figs'

# path to data
data_path= path + '/data'

# path to src folder
src_path = path + '/src'
print(src_path)

# sys path
sys.path.append(src_path)

/Users/mikey/LocalFiles/DS5230/final_project/DS5230-final/notebooks
/Users/mikey/LocalFiles/DS5230/final_project/DS5230-final
/Users/mikey/LocalFiles/DS5230/final_project/DS5230-final/src


____

### Import Modules

In [4]:
# modules and util
import dimensionality_reduction as dr
import clustering as cl
from cluster_utils import *
from external_indices_utils import *

___

### Parameters

In [5]:
# transformed data csv file name
data_file = "/curated/trans_data_design.csv"

# target vector csv file name
target_file = "/curated/beans_target.csv"

____

### Load Data

In [6]:
filename = '/curated/shuffled_target.csv'
target_vector_shuffled = pd.read_csv(data_path + filename)

filename = '/curated/shuffled_design.csv'
design_matrix_shuffled = pd.read_csv(data_path + filename)


____

### Sample Data

In [7]:
sample = True
if sample:
    # num samples
    n = 1000

    # random seed
    rand_seed = 42

    # sample target and save copy to csv
    target_vector_shuffled = target_vector_shuffled.sample(n=n, random_state=rand_seed)
    filename = '/curated/sampled_target.csv'
    target_vector_shuffled.to_csv(data_path + filename, index=False)

     # sample target and save copy to csv
    design_matrix_shuffled = design_matrix_shuffled.sample(n=n, random_state=rand_seed)
    filename = '/curated/sampled_design.csv'
    design_matrix_shuffled.to_csv(data_path + filename, index=False)

___

### Create capX

In [8]:
# drop ID col
design_matrix_shuffled_noID = design_matrix_shuffled.drop('ID', axis=1)

# convert to ndarray
cap_x = design_matrix_shuffled_noID.to_numpy()
print(f'cap_x shape: {cap_x.shape}')
cap_x

cap_x shape: (1000, 16)


array([[ 0.47951119,  0.75617026,  0.9025797 , ..., -1.08640545,
        -1.03819934, -2.53011841],
       [-0.49798027, -0.60106105, -0.58065225, ...,  0.38428313,
         0.26243519,  0.25945463],
       [-0.0963504 , -0.06761403, -0.09561878, ..., -0.10319885,
         0.13672302, -0.70899572],
       ...,
       [-0.30471913, -0.47210471, -0.71409026, ...,  1.3837794 ,
         1.95995551,  0.86201627],
       [ 3.61104419,  3.02864018,  2.64390167, ..., -1.24739667,
         0.33815956, -0.56965621],
       [ 0.00623113,  0.11297058,  0.20609565, ..., -0.56222324,
        -0.50146285, -0.33099767]])

___

### Select Hyperparameters for Gridsearch

In [9]:
# select hyperparams
params1=False
params2=True
params3=False
params4=False

if params1:

    min_dist_list = [0.0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.25, 0.5]
    n_neighbors_list = [10, 20, 30, 50, 100]
    metric_list =['euclidean', 'manhattan']
    n_components_list = [2, 3]


if params2:

    min_dist_list = [0.01, 0.05, 0.1]
    n_neighbors_list = [5, 10, 15]
    metric_list =['euclidean', 'manhattan']
    n_components_list = [2, 3]


if params3:

    min_dist_list = np.linspace(0, 0.5, num = 40)
    n_neighbors_list = np.arange(5, 35, 5)
    metric_list =['euclidean', 'manhattan']
    n_components_list = [2, 3]


if params4:

    min_dist_list = [0.01]
    n_neighbors_list = [10]
    metric_list =['euclidean']
    n_components_list = [2, 3]




___

### Gridsearch Algorithm

In [10]:
runs = len(n_components_list)*len(min_dist_list)*len(n_neighbors_list)*len(metric_list)
run = 0
df_row_dict_list = []
for n_components in n_components_list:
  for min_dist in min_dist_list:
    for n_neighbors in n_neighbors_list: 
      for metric in metric_list:
        run +=1
        print('*'*100)
        print(f'Run {run} of {runs}')
        results_dict = dr.umap_dim_red(cap_x, n_neighbors, min_dist, metric, n_components) 
        df_row_dict = cl.clustering(results_dict)
        df_row_dict_list.append(df_row_dict)
results_df = pd.DataFrame(df_row_dict_list)
results_df

****************************************************************************************************
Run 1 of 36


/Users/mikey/miniconda3/envs/usml_base_ds2/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


****************************************************************************************************
****************************************************************************************************
Hyperparameters:
n_neighbors:  5
min_dist:  0.01
metric:  euclidean
n_components:  2
Hopkin's Statistic = 0.12136865457220114
Fail: Kmeans did not successfully cluster.
DBSCAN
Number of Clusters:  11
Validity Index:  0.5419393459458002
****************************************************************************************************
Run 2 of 36


/Users/mikey/miniconda3/envs/usml_base_ds2/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


****************************************************************************************************
****************************************************************************************************
Hyperparameters:
n_neighbors:  5
min_dist:  0.01
metric:  manhattan
n_components:  2
Hopkin's Statistic = 0.14420927107337905
Fail: Kmeans did not successfully cluster.
DBSCAN
Number of Clusters:  51
Validity Index:  0.4652657691709719
****************************************************************************************************
Run 3 of 36


...

/Users/mikey/miniconda3/envs/usml_base_ds2/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


****************************************************************************************************
****************************************************************************************************
Hyperparameters:
n_neighbors:  15
min_dist:  0.1
metric:  euclidean
n_components:  3
Hopkin's Statistic = 0.12082531156447045
Fail: Kmeans did not successfully cluster.
DBSCAN
Number of Clusters:  5
Validity Index:  0.408608312348232
****************************************************************************************************
Run 36 of 36


/Users/mikey/miniconda3/envs/usml_base_ds2/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


****************************************************************************************************
****************************************************************************************************
Hyperparameters:
n_neighbors:  15
min_dist:  0.1
metric:  manhattan
n_components:  3
Hopkin's Statistic = 0.13099289478132645
Fail: Kmeans did not successfully cluster.
DBSCAN
Number of Clusters:  4
Validity Index:  0.6623422172262821


,algo,n_clusters_found,n_clusters_db_score_is_min,n_clusters_ch_score_is_max,n_clusters_silhouette_score_is_max,silhouette_score,hopkins_statistic,umap_n_neighbors,umap_min_dist,umap_metric,umap_n_components,trustworthiness,eps,dbscan_min_samples,validity_index,cluster_labels
0,dbscan,11,NaN,NaN,NaN,NaN,0.121369,5,0.01,euclidean,2,0.981220,0.641051,5,0.541939,"[0, 1, 2, 2, 1, 3, 1, 1, 0, 1, 0, 0, 1, 2, 1, ..."
1,dbscan,51,NaN,NaN,NaN,NaN,0.144209,5,0.01,manhattan,2,0.978543,0.266174,6,0.465266,"[0, 45, 1, 1, 8, 2, 3, 4, 0, 4, 5, 16, 6, 1, 7..."
2,dbscan,7,NaN,NaN,NaN,NaN,0.138859,10,0.01,euclidean,2,0.974305,0.473004,6,0.456507,"[0, 1, 2, 2, 1, 3, 1, 1, 0, 1, 0, 0, 1, 2, 1, ..."
3,dbscan,35,NaN,NaN,NaN,NaN,0.148817,10,0.01,manhattan,2,0.970405,0.230564,6,0.395595,"[0, 1, 2, 2, 3, 4, 5, 6, 0, 6, 7, 0, 8, 2, 5, ..."
4,dbscan,5,NaN,NaN,NaN,NaN,0.166864,15,0.01,euclidean,2,0.971605,0.448304,6,0.685085,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
5,dbscan,5,NaN,NaN,NaN,NaN,0.162733,15,0.01,manhattan,2,0.967436,0.522399,6,0.750654,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
6,dbscan,12,NaN,NaN,NaN,NaN,0.150849,5,0.05,euclidean,2,0.981524,0.572343,5,0.535316,"[0, 1, 2, 2, 1, 3, 1, 1, 0, 1, 0, 0, 1, 2, 1, ..."
7,dbscan,41,NaN,NaN,NaN,NaN,0.163376,5,0.05,manhattan,2,0.978425,0.298624,6,0.402230,"[0, -1, 1, 1, 9, 2, 3, 4, 0, 4, 5, 6, 7, 1, 8,..."
8,dbscan,7,NaN,NaN,NaN,NaN,0.166602,10,0.05,euclidean,2,0.978176,0.462389,6,0.492562,"[0, 1, 2, 2, 1, 3, 1, 1, 0, 1, 0, 0, 1, 2, 1, ..."
9,dbscan,4,NaN,NaN,NaN,NaN,0.188811,10,0.05,manhattan,2,0.970375,0.599744,6,0.749740,"[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."


___

## All Results

In [11]:
results_df.to_csv(f'../data/all_results_{str(datetime.now()).replace(" ", "")}.csv')

# display max rows
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
results_df

,algo,n_clusters_found,n_clusters_db_score_is_min,n_clusters_ch_score_is_max,n_clusters_silhouette_score_is_max,silhouette_score,hopkins_statistic,umap_n_neighbors,umap_min_dist,umap_metric,umap_n_components,trustworthiness,eps,dbscan_min_samples,validity_index,cluster_labels
0,dbscan,11,NaN,NaN,NaN,NaN,0.121369,5,0.01,euclidean,2,0.981220,0.641051,5,0.541939,"[0, 1, 2, 2, 1, 3, 1, 1, 0, 1, 0, 0, 1, 2, 1, ..."
1,dbscan,51,NaN,NaN,NaN,NaN,0.144209,5,0.01,manhattan,2,0.978543,0.266174,6,0.465266,"[0, 45, 1, 1, 8, 2, 3, 4, 0, 4, 5, 16, 6, 1, 7..."
2,dbscan,7,NaN,NaN,NaN,NaN,0.138859,10,0.01,euclidean,2,0.974305,0.473004,6,0.456507,"[0, 1, 2, 2, 1, 3, 1, 1, 0, 1, 0, 0, 1, 2, 1, ..."
3,dbscan,35,NaN,NaN,NaN,NaN,0.148817,10,0.01,manhattan,2,0.970405,0.230564,6,0.395595,"[0, 1, 2, 2, 3, 4, 5, 6, 0, 6, 7, 0, 8, 2, 5, ..."
4,dbscan,5,NaN,NaN,NaN,NaN,0.166864,15,0.01,euclidean,2,0.971605,0.448304,6,0.685085,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
5,dbscan,5,NaN,NaN,NaN,NaN,0.162733,15,0.01,manhattan,2,0.967436,0.522399,6,0.750654,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
6,dbscan,12,NaN,NaN,NaN,NaN,0.150849,5,0.05,euclidean,2,0.981524,0.572343,5,0.535316,"[0, 1, 2, 2, 1, 3, 1, 1, 0, 1, 0, 0, 1, 2, 1, ..."
7,dbscan,41,NaN,NaN,NaN,NaN,0.163376,5,0.05,manhattan,2,0.978425,0.298624,6,0.402230,"[0, -1, 1, 1, 9, 2, 3, 4, 0, 4, 5, 6, 7, 1, 8,..."
8,dbscan,7,NaN,NaN,NaN,NaN,0.166602,10,0.05,euclidean,2,0.978176,0.462389,6,0.492562,"[0, 1, 2, 2, 1, 3, 1, 1, 0, 1, 0, 0, 1, 2, 1, ..."
9,dbscan,4,NaN,NaN,NaN,NaN,0.188811,10,0.05,manhattan,2,0.970375,0.599744,6,0.749740,"[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."


In [12]:
# set max rows back to default
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

____

### Get 10 Best Results (Kmenas or DBSCAN)

In [13]:
results_df['combined_score'] = results_df['silhouette_score'].fillna(results_df['validity_index'])
results_df = results_df.sort_values(by='combined_score', ascending=False)
filter = results_df.head(10)
filtered_results_df = results_df[results_df.index.isin(filter.index)]
filtered_results_df = filtered_results_df.drop(columns=['combined_score'])

filtered_results_df

,algo,n_clusters_found,n_clusters_db_score_is_min,n_clusters_ch_score_is_max,n_clusters_silhouette_score_is_max,silhouette_score,hopkins_statistic,umap_n_neighbors,umap_min_dist,umap_metric,umap_n_components,trustworthiness,eps,dbscan_min_samples,validity_index,cluster_labels
5,dbscan,5,NaN,NaN,NaN,NaN,0.162733,15,0.01,manhattan,2,0.967436,0.522399,6,0.750654,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
9,dbscan,4,NaN,NaN,NaN,NaN,0.188811,10,0.05,manhattan,2,0.970375,0.599744,6,0.749740,"[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
23,dbscan,5,NaN,NaN,NaN,NaN,0.088093,15,0.01,manhattan,3,0.967690,0.465981,6,0.742121,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
29,dbscan,5,NaN,NaN,NaN,NaN,0.099274,15,0.05,manhattan,3,0.968684,0.453951,5,0.686687,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
4,dbscan,5,NaN,NaN,NaN,NaN,0.166864,15,0.01,euclidean,2,0.971605,0.448304,6,0.685085,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
22,dbscan,5,NaN,NaN,NaN,NaN,0.091115,15,0.01,euclidean,3,0.972572,0.543401,6,0.681348,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
21,dbscan,5,NaN,NaN,NaN,NaN,0.084507,10,0.01,manhattan,3,0.970933,0.653201,6,0.670477,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
35,dbscan,4,NaN,NaN,NaN,NaN,0.130993,15,0.10,manhattan,3,0.967873,0.689014,5,0.662342,"[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
28,dbscan,5,NaN,NaN,NaN,NaN,0.100366,15,0.05,euclidean,3,0.973940,0.569202,6,0.636526,"[0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."
15,dbscan,4,NaN,NaN,NaN,NaN,0.216245,10,0.10,manhattan,2,0.970116,0.535237,5,0.614100,"[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, ..."


____

### Save Filtered Results to CSV

In [ ]:
filename = f'/curated/filtered_results_{str(datetime.now()).replace(" ", "")}.csv'
filtered_results_df.to_csv(data_path + filename, index=False)

___

## Runtime

In [14]:
finish = time.time()
hours = int((finish - start) // 3600)
minutes = int(((finish - start) % 3600) // 60)
seconds = int((finish - start) % 60)
print(f"Total Run Time(hh:mm.ss): {hours:02d}:{minutes:02d}.{seconds:02d}")

Total Run Time(hh:mm.ss): 00:02.13
